In [ ]:
# Environment secret keys
from google.colab import userdata
import os
from langchain_community.retrievers import WikipediaRetriever
from langchain.chains import RetrievalQA
from langchain_huggingface import HuggingFaceEndpoint

# Retrieve API token securely
sec_key = userdata.get("HUGGINGFACEHUB_API_TOKEN")

# Ensure the API key exists
if not sec_key:
    raise ValueError("HUGGINGFACEHUB_API_TOKEN not found! Make sure it is set in Colab.")

# Set the environment variable
os.environ["HUGGINGFACEHUB_API_TOKEN"] = sec_key

# Initialize Wikipedia Retriever
wikipedia = WikipediaRetriever()

# Initialize Hugging Face Model
repo_id = "deepseek-ai/DeepSeek-R1"
llm = HuggingFaceEndpoint(
    repo_id=repo_id,
    task="text-generation",
    max_length=512,
    min_length=250,
    temperature=0.9
)

def generate_news_article(topic):
    """
    Generates a well-structured news article using RAG (Wikipedia + LLM).
    """

    # Retrieve relevant Wikipedia content
    relevant_docs = wikipedia.get_relevant_documents(topic)
    if not relevant_docs:
        raise ValueError(f"No Wikipedia content found for topic: {topic}")

    context = "\n\n".join([doc.page_content for doc in relevant_docs[:3]])  # Limit to 3 relevant docs

    # Construct prompt with Wikipedia context
    prompt = f"""
    You are a professional journalist. Write a well-structured news article on '{topic}' in three paragraphs.

    *Use the provided information from Wikipedia:*
    {context}

    *Requirements:*
    1️⃣ Start with a **catchy and engaging headline**.
    2️⃣ *First paragraph* → Introduce the topic with a compelling hook, latest news and  explain why it matters globally.
    3️⃣ *Second paragraph* → Provide key details, background, and industry impact.
    4️⃣ *Third paragraph* → Conclude with the broader implications and possible future developments.

    Format the article in a continuous flow without subheadings or section markers.
    """

    response = llm.invoke(prompt)
    return response


                    max_length was transferred to model_kwargs.
                    Please make sure that max_length is what you intended.
                    min_length was transferred to model_kwargs.
                    Please make sure that min_length is what you intended.


In [ ]:
# Function to format paragraphs with new line after every 15 tokens and 2 line spaces between paragraphs
def format_paragraphs(news_article):
    paragraphs = news_article.strip().split('\n')
    formatted_output = ""

    for paragraph in paragraphs:
        words = paragraph.split()  # Tokenize into words
        formatted_paragraph = ""

        # Add new line after every 15 words
        for i in range(0, len(words), 15):
            formatted_paragraph += " ".join(words[i:i+15]) + "\n"

        # Add two line spaces between paragraphs
        formatted_output += formatted_paragraph + "\n"

    return formatted_output


In [ ]:

# Get formatted output
topic = "Ola Kaellenius"
news_article = generate_news_article(topic)
formatted_article = format_paragraphs(news_article)

# Print the result
print("\nGenerated Newspaper Article:\n")
print(formatted_article)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



Generated Newspaper Article:

Examples of newspapers: [The Guardian](https://www.theguardian.com/international), [BBC News](https://www.bbc.com/news/world), [The New York Times](https://www.nytimes.com/international/)



Headline:

Ola Kaellenius: The Steward of Mercedes-Benz's Alabama Plant and Its Evolution


The man behind the wheel of Mercedes-Benz's largest SUV plant in the world, Ola Kaellenius,
has been instrumental in the Alabama facility's growth and diversification. Kaellenius, a Swede by birth,
took the helm of Mercedes-Benz U.S. International (MBUSI) in 2009, succeeding Bill Taylor. His tenure
coincided with a significant shift in the plant's production portfolio, including the introduction of the
C-Class and the move towards SUVs and electric vehicles.


Under Kaellenius's leadership, MBUSI expanded its offerings to include not only the popular GLE, GLE
Coupe, and GLS models but also the high-end Mercedes-Maybach GLS. More recently, the plant started
production of the EQE

In [ ]:
!jupyter nbconvert --to html Article_Generator.ipynb

[NbConvertApp] Converting notebook Article_Generator.ipynb to html
[NbConvertApp] Writing 330681 bytes to Article_Generator.html
